In [15]:
import pandas as pd
import torch
import numpy as np
from scipy.stats import spearmanr
device = "cuda" if torch.cuda.is_available() else "cpu"

In [16]:
log = pd.read_pickle("../backup/results_full_32_255.pkl")

log = {
        'img_id': [],
        'interpretability_method': [],
        'label': [],
        'similarity_original': [],
        'similarity_perturb': [],
        'hm_original': [],
        'hm_perturbed': [],
        'alpha': [],
        'clip_model': []
    }

In [20]:
log.tail()

,img_id,interpretability_method,label,similarity_original,similarity_perturb,hm_original,hm_perturbed,alpha,clip_model,spearman_rank_correlation,top_k_intersection,k_value,ssim,simimilarity_diff
47569,5284,eclip,0,0.27002,0.317627,"[[0.001682, 0.001682, 0.001682, 0.001682, 0.00...","[[0.0726, 0.0726, 0.0726, 0.0726, 0.0726, 0.07...",0.12549,ViT-B/16,NaN,NaN,NaN,NaN,NaN
47570,5285,selfattn,0,0.25708,0.311768,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.12549,ViT-B/16,NaN,NaN,NaN,NaN,NaN
47571,5285,gradcam,0,0.25708,0.311768,"[[0.2375, 0.2375, 0.2375, 0.2375, 0.2375, 0.23...","[[0.10535, 0.10535, 0.10535, 0.10535, 0.10535,...",0.12549,ViT-B/16,NaN,NaN,NaN,NaN,NaN
47572,5285,maskclip,0,0.25708,0.311768,"[[0.2544, 0.2544, 0.2544, 0.2544, 0.2544, 0.25...","[[0.2131, 0.2131, 0.2131, 0.2131, 0.2131, 0.21...",0.12549,ViT-B/16,NaN,NaN,NaN,NaN,NaN
47573,5285,eclip,0,0.25708,0.311768,"[[0.0313, 0.0313, 0.0313, 0.0313, 0.0313, 0.03...","[[0.0309, 0.0309, 0.0309, 0.0309, 0.0309, 0.03...",0.12549,ViT-B/16,NaN,NaN,NaN,NaN,NaN


In [18]:
log = log.drop(log[log['interpretability_method'].str.contains('eclip|selfattn|gradcam|maskclip', na=False)].index)

In [19]:
novo_df = pd.read_pickle("../backup/results_corrected_32_255.pkl")
log = pd.concat([log, novo_df], ignore_index=True)

In [21]:
log.to_pickle("../backup/results_full_32_255.pkl")

In [8]:
filtered_data = log.loc[log["interpretability_method"] == "eclip", ["hm_original", "hm_perturbed"]]
filtered_data

,hm_original,hm_perturbed
3,"[[0.06186527, 0.06186527, 0.06186527, 0.061865...","[[0.06186527, 0.06186527, 0.06186527, 0.061865..."


In [ ]:
for idx, row in filtered_data.iterrows():
    # Cada heatmap está como lista de listas de tensores -> converter para tensor
    orig = torch.tensor([[t.item() for t in inner] for inner in row["hm_original"]])
    pert = torch.tensor([[t.item() for t in inner] for inner in row["hm_perturbed"]])
    
    # Flatten para 1D arrays
    flat_orig = orig.flatten().to(device).numpy()
    flat_perturb = pert.flatten().to(device).numpy()
    
    # Calcular Spearman
    corr, p_val = spearmanr(flat_orig, flat_perturb)
    print(f"Spearman correlation: {corr:.3f}, p-value: {p_val:.3e}")

In [ ]:
def topk_intersection(map1, map2, k):
    flat1 = map1.flatten()
    flat2 = map2.flatten()
    
    topk_indices_1 = torch.topk(flat1.clone().detach(), k).indices
    topk_indices_2 = torch.topk(flat2.clone().detach(), k).indices

    set1 = set(topk_indices_1.tolist())
    set2 = set(topk_indices_2.tolist())    
    
    intersection = set1.intersection(set2)
    
    return len(intersection), len(intersection) / k

# Valor de k (pode ajustar)
k = 100

for idx, row in filtered_data.iterrows():
    # Converter para tensores
    map1 = torch.tensor([[t.item() for t in inner] for inner in row["hm_original"]])
    map2 = torch.tensor([[t.item() for t in inner] for inner in row["hm_perturbed"]])
    
    # Calcular interseção
    inter_count, inter_ratio = topk_intersection(map1, map2, k)
    print(f"Top-{k} intersection count: {inter_count}")
    print(f"Top-{k} intersection ratio: {inter_ratio:.3f}") 

In [15]:
from skimage.metrics import structural_similarity as ssim

hm_original_values = []
hm_perturbed_values = []

for idx, row in log.iterrows():
    # Obter os tensores de hm_original e hm_perturbed
    hm_original = np.array([tensor.flatten() for tensor in row['hm_original']])  # Transformar em array 1D
    hm_perturbed = np.array([tensor.flatten() for tensor in row['hm_perturbed']])  # Transformar em array 1D
    
    # Certificar-se de que os dados estão no formato adequado para o SSIM
    # Reshaping para 2D (precisa ser 2D para o cálculo do SSIM)
    hm_original = hm_original.reshape(224, 224)  # Exemplo, ajuste conforme o tamanho real do mapa
    hm_perturbed = hm_perturbed.reshape(224, 224)

    ssim_value = ssim(hm_original, hm_perturbed, data_range=hm_perturbed.max() - hm_perturbed.min())
    print(f"Method: {row['interpretability_method']}\nssim value: {ssim_value} ")



Method: selfattn
ssim value: 1.0 
Method: gradcam
ssim value: 1.0 
Method: maskclip
ssim value: 1.0 
Method: eclip
ssim value: 1.0 
Method: game
ssim value: 0.41159949828650183 
Method: rollout
ssim value: 0.7008852546549125 
Method: surgery
ssim value: 0.5809520827451814 
Method: m2ib
ssim value: 0.4870703768018727 
Method: rise
ssim value: 0.38206883497691096 


In [28]:
agg_data = log.groupby('img_id').agg({
    'similarity_original': 'mean',  # Média das similaridades originais
    'similarity_perturb': 'mean',   # Média das similaridades perturbadas
}).reset_index()

for idx, row in agg_data.iterrows():
    sim_diff = (row['similarity_perturb'] - row['similarity_original']) / row['similarity_original']
    print(f"image id: {row['img_id']}\nsimilarity difference: {sim_diff} ")
    

image id: 0.0
similarity difference: 0.023791759128641756 


In [23]:
agg_data = log.groupby('img_id').agg({
    'similarity_original': 'mean',  # Média das similaridades originais
    'similarity_perturb': 'mean',   # Média das similaridades perturbadas
}).reset_index()

print(agg_data)

   img_id  similarity_original  similarity_perturb
0       0             0.297058            0.304125
